In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 4

def train_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
# #     inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
#     inp = [numpy.dstack([scene['p_in']]) for scene in batch]
# #     out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
#     out = [numpy.dstack([scene['p_out']]) for scene in batch]
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:], scene['v_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
#     out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    out = [numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:], scene['v_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:], scene['v_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=train_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = False, collate_fn=test_collate, num_workers=0)

### Visualize the batch of sequences

In [4]:
import matplotlib.pyplot as plt
import random

agent_id = 0

def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a randon agent"""
    inp, out = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1])
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1])

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EncoderRNN(nn.Module):
    """Encoder Network."""
    def __init__(self,
                 input_size: int = 2,
                 embedding_size: int = 8,
                 hidden_size: int = 16):
        """Initialize the encoder network.

        Args:
            input_size: number of features in the input
            embedding_size: Embedding size
            hidden_size: Hidden size of LSTM

        """
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(input_size, embedding_size)
        self.lstm1 = nn.LSTMCell(embedding_size, hidden_size)

    def forward(self, x: torch.FloatTensor, hidden: Any) -> Any:
        """Run forward propagation.

        Args:
            x: input to the network
            hidden: initial hidden state
        Returns:
            hidden: final hidden 

        """
        embedded = F.relu(self.linear1(x))
        hidden = self.lstm1(embedded, hidden)
        return hidden


class DecoderRNN(nn.Module):
    """Decoder Network."""
    def __init__(self, embedding_size=8, hidden_size=16, output_size=2):
        """Initialize the decoder network.

        Args:
            embedding_size: Embedding size
            hidden_size: Hidden size of LSTM
            output_size: number of features in the output

        """
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm1 = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        """Run forward propagation.

        Args:
            x: input to the network
            hidden: initial hidden state
        Returns:
            output: output from lstm
            hidden: final hidden state

        """
        embedded = F.relu(self.linear1(x))
        hidden = self.lstm1(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden

In [7]:
from tqdm import tqdm_notebook as tqdm

def train(
        train_loader: Any,
        epoch: int,
        criterion: Any,
        logger: Logger,
        encoder: Any,
        decoder: Any,
        encoder_optimizer: Any,
        decoder_optimizer: Any,
        model_utils: ModelUtils,
        rollout_len: int = 30,
) -> None:
    """Train the lstm network.

    Args:
        train_loader: DataLoader for the train set
        epoch: epoch number
        criterion: Loss criterion
        logger: Tensorboard logger
        encoder: Encoder network instance
        decoder: Decoder network instance
        encoder_optimizer: optimizer for the encoder network
        decoder_optimizer: optimizer for the decoder network
        model_utils: instance for ModelUtils class
        rollout_len: current prediction horizon

    """
    args = parse_arguments()
    global global_step

    for i, (_input, target, helpers) in enumerate(train_loader):
        _input = _input.to(device)
        target = target.to(device)

        # Set to train mode
        encoder.train()
        decoder.train()

        # Zero the gradients
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = model_utils.init_hidden(
            batch_size,
            encoder.module.hidden_size if use_cuda else encoder.hidden_size)

        # Initialize losses
        loss = 0

        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(target.shape).to(device)

        # Decode hidden state in future trajectory
        for di in range(rollout_len):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Update loss
            loss += criterion(decoder_output[:, :2], target[:, di, :2])

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / rollout_len

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()

        if global_step % 1000 == 0:

            # Log results
            print(
                f"Train -- Epoch:{epoch}, loss:{loss}, Rollout:{rollout_len}")

            logger.scalar_summary(tag="Train/loss",
                                  value=loss.item(),
                                  step=epoch)

        global_step += 1

In [8]:
def predict(model, device, test_loader):
    model.eval()
    counter = 0
    all_outs = numpy.zeros((1,60))
    idx_out = []
#     print(all_outs.shape)
    with torch.no_grad():
        for data, idx in test_loader:
            data= data[:,0,:,:]
            data= data.to(device)
            output = model(data)
            output = output[:,:,0:2].reshape(-1,60)
            arr = output.data.cpu().numpy()
#             print(arr.shape)
            all_outs = numpy.append(all_outs, arr, axis=0)
#             print(all_outs)
#             print(idx)
#             print('')
            idx_out.append(int(idx[0][0][0][0]))
            idx_out.append(int(idx[1][0][0][0]))
            idx_out.append(int(idx[2][0][0][0]))
            idx_out.append(int(idx[3][0][0][0]))
#     print(all_outs.shape)
#     print(idx_out[0])
#     print(len(idx_out))
    all_outs = all_outs[1:,:]
#     all_outs = numpy.insert(all_outs, 0, idx_out, axis=1)
#     for i in range(len(all_outs)):
#         print(all_outs[i][0])
#         all_outs[i][0] = idx_out[i]
#     print(all_outs.shape)\
    print(all_outs)
    return all_outs, idx_out

In [9]:
device = "cuda"
model = NN().to(device) #using gpu here
optimizer = optim.Adagrad(model.parameters(), lr=0.01, lr_decay=0, weight_decay=0, initial_accumulator_value=0)
num_epoch = 1

for epoch in range(1, num_epoch + 1):
        train(model, device, val_loader, optimizer, epoch)
#         predict(model, device, test_loader)

KeyboardInterrupt: 

In [16]:
# imports
import numpy as np
import pandas as pd
from typing import Any, Dict, List, Tuple, Optional, Union

# main from lstm_train_test.py

In [4]:
# https://github.com/jagjeet-singh/argoverse-forecasting/blob/master/lstm_train_test.py

# $ python lstm_train_test.py 
# --train_features <path/to/train/features> 
# --val_features <path/to/val/features> 
# --test_features <path/to/test/features> 
# --model_path <path/to/saved/checkpoint> 
# --use_delta 
# --normalize 
# --obs_len 20 
# --pred_len 30 
# --model_path <pkl/file/path/for/model> 
# --traj_save_path <pkl/file/for/forecasted/trajectories>

# training set folder path
train_folder_path = "./new_train/new_train"

# no validation set
val_folder_path = None

# test set folder path
test_folder_path = './new_val_in/new_val_in'

# model path (pick up from where you left off in training)
model_path = None

# model pickl (TODO: need to find out what this means)
model_pkl = None

# delta (training on change in position rather than absolute position)
delta = True

# normalize (TODO: need to find out what this means)
normalize = True

# obs_len (train on given number of steps)
obs_len = 19

# pred_len (predict number of steps)
pred_len = 30

# trajectory save path (where we save our predicted path)
traj_path = "./trajectory.pkl"

## use cuda


In [5]:
device = "cuda"

# TODO match this to lstm train test

## modelutils

In [10]:
class ModelUtils:
    """Utils for LSTM baselines."""
    def save_checkpoint(self, save_dir: str, state: Dict[str, Any]) -> None:
        """Save checkpoint file.
        
        Args:
            save_dir: Directory where model is to be saved
            state: State of the model

        """
        filename = "{}/LSTM_rollout{}.pth.tar".format(save_dir,
                                                      state["rollout_len"])
        torch.save(state, filename)

    def load_checkpoint(
            self,
            checkpoint_file: str,
            encoder: Any,
            decoder: Any,
            encoder_optimizer: Any,
            decoder_optimizer: Any,
    ) -> Tuple[int, int, float]:
        """Load the checkpoint.

        Args:
            checkpoint_file: Path to checkpoint file
            encoder: Encoder model
            decoder: Decoder model 

        Returns:
            epoch: epoch when the model was saved.
            rollout_len: horizon used
            best_loss: loss when the checkpoint was saved

        """
        if os.path.isfile(checkpoint_file):
            print("=> loading checkpoint '{}'".format(checkpoint_file))
            checkpoint = torch.load(checkpoint_file)
            epoch = checkpoint["epoch"]
            best_loss = checkpoint["best_loss"]
            rollout_len = checkpoint["rollout_len"]
            if use_cuda:
                encoder.module.load_state_dict(
                    checkpoint["encoder_state_dict"])
                decoder.module.load_state_dict(
                    checkpoint["decoder_state_dict"])
            else:
                encoder.load_state_dict(checkpoint["encoder_state_dict"])
                decoder.load_state_dict(checkpoint["decoder_state_dict"])
            encoder_optimizer.load_state_dict(checkpoint["encoder_optimizer"])
            decoder_optimizer.load_state_dict(checkpoint["decoder_optimizer"])
            print(
                f"=> loaded checkpoint {checkpoint_file} (epoch: {epoch}, loss: {best_loss})"
            )
        else:
            print(f"=> no checkpoint found at {checkpoint_file}")

        return epoch, rollout_len, best_loss

    def my_collate_fn(self, batch: List[Any]) -> List[Any]:
        """Collate function for PyTorch DataLoader.

        Args:
            batch: Batch data

        Returns: 
            input, output and helpers in the format expected by DataLoader

        """
        _input, output, helpers = [], [], []

        for item in batch:
            _input.append(item[0])
            output.append(item[1])
            helpers.append(item[2])
        _input = torch.stack(_input)
        output = torch.stack(output)
        return [_input, output, helpers]

    def init_hidden(self, batch_size: int,
                    hidden_size: int) -> Tuple[Any, Any]:
        """Get initial hidden state for LSTM.

        Args:
            batch_size: Batch size
            hidden_size: Hidden size of LSTM

        Returns:
            Initial hidden states

        """
        return (
            torch.zeros(batch_size, hidden_size).to(device),
            torch.zeros(batch_size, hidden_size).to(device),
        )

In [11]:
model_utils = ModelUtils()

## get data function

In [20]:
def get_data(test_features, train_features) -> Dict[str, Union[np.ndarray, pd.DataFrame, None]]:
    """Load data from local data_dir.

    Args:
        args (argparse): Arguments to baseline
        baseline_key: Key for obtaining features for the baseline
    Returns:
        data_dict (dict): Dictionary of input/output data and helpers for train/val/test splits

    """
    input_features = ["X", "Y"]
    output_features = ["X", "Y"]
    if test_features:
        print("Loading Test data ...")
        test_input, test_output, test_df = load_and_preprocess_data(
            input_features,
            output_features,
            test_features,
            mode="test")
        print("Test Size: {}".format(test_input.shape[0]))
    else:
        test_input, test_output, test_df = [None] * 3

    if train_features:
        print("Loading Train data ...")
        train_input, train_output, train_df = load_and_preprocess_data(
            input_features,
            output_features,
            args,
            args.train_features,
            mode="train")
        print("Train Size: {}".format(train_input.shape[0]))
    else:
        train_input, train_output, train_df = [None] * 3

    data_dict = {
        "train_input": train_input,
        "test_input": test_input,
        "train_output": train_output,
        "test_output": test_output,
        "train_helpers": train_df,
        "test_helpers": test_df,
    }

    return data_dict


## load and preprocess data function

In [ ]:
def load_and_preprocess_data(
        input_features: List[str],
        output_features: List[str],
        feature_file: str,
        mode: str = "train",
) -> Tuple[np.ndarray, np.ndarray, pd.DataFrame]:
    """Load the data and preprocess based on given arguments.

    Args:
        input_features (list of str): Input features for the baseline
        output_features (list of str): Output features for the baseline
        args (argparse): Arguments to runNNBaselines.py/runLSTMBaselines.py
        feature_file: path to the file containing features
        mode (str): train/val/test
    Returns:
        _input: Input to the baseline
        _output: Ground truth 
        df: Helper values useful in visualization and evaluation

    """
    df = pd.read_pickle(feature_file)

    # Normalize if its a non-map baseline
    if not args.use_map and args.normalize:

        print("Normalizing ...")

        # Don't use X,Y as features
        input_feature_idx = [
            FEATURE_FORMAT[feature] for feature in input_features
            if feature != "X" and feature != "Y"
        ]
        output_feature_idx = [
            FEATURE_FORMAT[feature] for feature in output_features
            if feature != "X" and feature != "Y"
        ]

        # Normalize the trajectory
        normalized_traj_arr = get_normalized_traj(df, args)

        # Get other features
        input_features_data = np.stack(
            df["FEATURES"].values)[:, :, input_feature_idx].astype("float")
        output_features_data = np.stack(
            df["FEATURES"].values)[:, :, output_feature_idx].astype("float")

        # Merge normalized trajectory and other features
        input_features_data = np.concatenate(
            (normalized_traj_arr, input_features_data), axis=2)
        output_features_data = np.concatenate(
            (normalized_traj_arr, output_features_data), axis=2)

    else:

        input_feature_idx = [
            FEATURE_FORMAT[feature] for feature in input_features
        ]
        output_feature_idx = [
            FEATURE_FORMAT[feature] for feature in output_features
        ]

        input_features_data = np.stack(
            df["FEATURES"].values)[:, :, input_feature_idx].astype("float")
        output_features_data = np.stack(
            df["FEATURES"].values)[:, :, output_feature_idx].astype("float")

    # If using relative distance instead of absolute
    # Store the first coordinate (reference) of the trajectory to map it back to absolute values later
    if args.use_delta:

        # Get relative distances for all topk centerline candidates
        if args.use_map and mode == "test":

            print("Creating relative distances for candidate centerlines...")

            # Relative candidate distances nt
            candidate_nt_distances = df["CANDIDATE_NT_DISTANCES"].values
            candidate_references = []
            for candidate_nt_dist_i in candidate_nt_distances:
                curr_reference = []
                for curr_candidate_nt in candidate_nt_dist_i:
                    curr_candidate_reference = get_relative_distance(
                        np.expand_dims(curr_candidate_nt, 0), mode, args)
                    curr_candidate_nt = curr_candidate_nt.squeeze()
                    curr_reference.append(curr_candidate_reference.squeeze())
                candidate_references.append(curr_reference)

            df["CANDIDATE_DELTA_REFERENCES"] = candidate_references

        else:

            print("Creating relative distances...")

            # Relative features
            reference = get_relative_distance(input_features_data, mode, args)
            _ = get_relative_distance(output_features_data, mode, args)
            df["DELTA_REFERENCE"] = reference.tolist()

    # Set train and test input/output data
    _input = input_features_data[:, :args.obs_len]

    if mode == "test":
        _output = None
    else:
        _output = output_features_data[:, args.obs_len:]

    return _input, _output, df

## get data_dict

In [22]:
data_dict = get_data(
    test_features = test_folder_path,
)

NameError: name 'args' is not defined